In [2]:
# ==================================================
# Azure Blob Storage Demo
# ==================================================
import sys, os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("."))

from app.clients.blob_storage import BlobStorageClient
# Initialisierung
client = BlobStorageClient()

# Testdatei Erzeugung
test_file_path = "test_upload.txt"
with open(test_file_path, "w") as f:
    f.write("Testinhalt - Azure Blob Storage funktioniert!")

# Datei hochladen
upload_ok = client.upload_file(test_file_path, "upload_test_notebook.txt")
print("Upload erfolgreich: ", upload_ok)

# Dateien herunterladen
download_path = "download_test_notebook.txt"
download_ok = client.download_file("upload_test_notebook.txt", download_path)
print ("/nDownload erfolgreich: ", download_ok)

# Lokalen Inhalt Überprüfung
with open(download_path, "r") as f:
    content = f.read()
print("/nInhalt der heruntergeladenen Dateien: ")
print(content)

Upload erfolgreich:  True
/nDownload erfolgreich:  True
/nInhalt der heruntergeladenen Dateien: 
Testinhalt - Azure Blob Storage funktioniert!


In [3]:
# =========================================================
# Löschung vom bestehenden Index
# =========================================================
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchableField

from app.config import AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_KEY, AZURE_SEARCH_INDEX


index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

# Wenn alter Index vorhanden ist, es wird zuerst gelöscht
try:
    index_client.delete_index(AZURE_SEARCH_INDEX)
    print(f"Alter Index ({AZURE_SEARCH_INDEX}) wurde gelöscht.")
except Exception:
    pass



Alter Index (documents) wurde gelöscht.


In [4]:
# ===========================================================
# Erstellung eines neuen Indexes
# ============================================================
fields = [
    SimpleField(name="id", type="Edm.String", key=True),
    SearchableField(name="title", type="Edm.String", retrievable=True),
    SearchableField(name="content", type="Edm.String", retrievable=True)
]

index = SearchIndex(name=AZURE_SEARCH_INDEX, fields=fields)

try:
    index_client.create_index(index)
    print(f"Index '{AZURE_SEARCH_INDEX}' erfolgreich initialisiert.")
except Exception as e:
    print(f"Fehler ist bei der Erstellung des Indexes aufgetreten: {e}")

Index 'documents' erfolgreich initialisiert.


In [5]:
# ========================================================
# Das Hochladen von Beispieldokumenten
# ========================================================
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_KEY)
)

docs = [
     {"id": "1", "title": "Azure Cognitive Search Test", "content": "Dies ist ein Testdokument."},
    {"id": "2", "title": "Künstliche Intelligenz", "content": "Azure Search ermöglicht KI-basierte Suche."},
    {"id": "3", "title": "Cloud Computing", "content": "Speicherung und Verarbeitung in der Cloud."},
    {"id": "4", "title": "Python Programmierung", "content": "Python ist eine beliebte Programmiersprache."},
    {"id": "5", "title": "Azure Einsatz", "content": "Azure könnte tatsächlich bei Firmen auf verschiedenen Wegen genutzt werden. In erster Linie ist es hervorzuheben, dass Azure Cognitive Services einen großen Mehrwert bei zahlreichen heikligen Aufgaben verschaffen kann."},
    {"id": "6", "title": "FastAPI", "content": "Erstellung von APIs mit FastAPI."}
]

search_client.upload_documents(documents=docs)
print("Dokumente erfolgreich hochgeladen.")

Dokumente erfolgreich hochgeladen.


In [6]:
# ========================================================
# Cognitive Search Demo
# =========================================================
from app.clients.cognitive_search import CoginitiveSearchClient
from app.config import AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_KEY, AZURE_SEARCH_INDEX

# Initialisierung des Clients
try:
    search_client = CoginitiveSearchClient()
    print("Cognitive Search Client wurde erfolgreich initialisert.")
except Exception as e:
    print("Fehler bei der Initialisierung: {e}")

query = "Azure"

try:
    results = search_client.search_documents(query=query, top=5)
    print(f"Gefundene Dokumente: {len(results)}")
    for doc in results:
        print(doc)
except Exception as e:
    print(f"Fehler bei der Suche: {e}")

Cognitive Search Client wurde erfolgreich initialisert.
Gefundene Dokumente: 3
{'content': 'Azure könnte tatsächlich bei Firmen auf verschiedenen Wegen genutzt werden. In erster Linie ist es hervorzuheben, dass Azure Cognitive Services einen großen Mehrwert bei zahlreichen heikligen Aufgaben verschaffen kann.', 'id': '5', 'title': 'Azure Einsatz', '@search.score': 1.6750907, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'content': 'Azure Search ermöglicht KI-basierte Suche.', 'id': '2', 'title': 'Künstliche Intelligenz', '@search.score': 1.133612, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'content': 'Dies ist ein Testdokument.', 'id': '1', 'title': 'Azure Cognitive Search Test', '@search.score': 0.7648603, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}


In [2]:
# ========================================
# OpenAI Modell Aufruf Demo
# ========================================
import sys
import os 

sys.path.append(os.path.abspath(".."))

from app.clients.openai_client import OpenAIClientWrapper

client = OpenAIClientWrapper()
response = client.generate_answer("Was ist Azure OpenAI?", max_tokens=1000)
print(response)

Azure OpenAI ist ein Cloud-Dienst von Microsoft, der es Entwicklern und Unternehmen ermöglicht, KI-Modelle von OpenAI über die Azure-Plattform zu nutzen. Diese Integration bringt die leistungsstarken Sprach- und KI-Modelle von OpenAI, wie GPT-3 und andere, in die Azure-Cloud-Umgebung, sodass Benutzer diese Technologien für ihre eigenen Anwendungen und Dienste einsetzen können.

Einige der Hauptmerkmale von Azure OpenAI sind:

1. **Skalierbarkeit**: Durch die Nutzung von Azure können Unternehmen die Rechenressourcen nach Bedarf skalieren.

2. **Sicherheit und Compliance**: Microsoft bietet verschiedene Sicherheits- und Compliance-Standards, die Unternehmen helfen, ihre Daten zu schützen und regulatorische Anforderungen zu erfüllen.

3. **Einfache Integration**: Azure OpenAI lässt sich gut in andere Azure-Dienste, wie Azure Functions, Azure Logic Apps und Azure Cognitive Services, integrieren.

4. **Zugänglichkeit**: Entwickler können über einfache API-Schnittstellen auf die KI-Funktione

In [4]:
# ==============================================================
# RAG Pipeline (Azure KI Suche) mit dem Modell Integration: Demo
# ==============================================================
from app.clients.cognitive_search import CoginitiveSearchClient
from app.clients.openai_client import OpenAIClientWrapper

# Initialisierung von Clients
search_client = CoginitiveSearchClient()
openai_client = OpenAIClientWrapper()

# Suchanfrage in Azure KI Suche
query = "Azure"
docs = search_client.search_documents(query=query, top=3)

# Antwort von GPT basierend auf Dokumenten wird erzeugt
prompt = "Erkläre mir, wie Azure in Unternehmen eingesetzt werden kann."
response = openai_client.generate_answer(prompt, context_docs=docs)
print(response)

Azure kann in Unternehmen auf verschiedene Weise eingesetzt werden, um deren Effizienz und Innovationsfähigkeit zu steigern. Hier sind einige wichtige Einsatzmöglichkeiten:

1. **Cognitive Services**: Azure Cognitive Services bieten eine Reihe von KI-gestützten APIs, die Unternehmen helfen, komplexe Aufgaben zu bewältigen. Diese Dienste ermöglichen es, natürliche Sprache zu verarbeiten, Bilder und Videos zu analysieren und sogar Entscheidungen auf der Grundlage von Daten zu treffen. Beispielsweise können Unternehmen Chatbots erstellen, die Kundenanfragen automatisiert bearbeiten, oder Bildanalysesysteme zur Qualitätskontrolle in der Produktion nutzen.

2. **KI-basierte Suche mit Azure Search**: Azure Search ist ein leistungsfähiges Tool, das es Unternehmen ermöglicht, eine KI-unterstützte Suchfunktion in ihren Anwendungen zu integrieren. Dies erleichtert das Finden relevanter Informationen innerhalb großer Datenmengen und verbessert die Benutzererfahrung sowohl für interne als auch für